In this chapter we used already fine-tuned BERT models to extract the chunk embeddings of our books. The chunk embeddings correspond to either the meaned embeddings of all the words in the sequence or the embedding of the [CLS] token. We will explore the results of both. We will then run a variety of classifiers over these embeddings directly. 

1.   Meaned pooled output --> single layer NN
2.   RoBERT
3.   ToBERT ?

# Installs, Imports, Configuration

In [1]:
!pip install datasets

     |████████████████████████████████| 542 kB 5.5 MB/s 
     |████████████████████████████████| 118 kB 20.7 MB/s 
     |████████████████████████████████| 76 kB 4.0 MB/s 
     |████████████████████████████████| 243 kB 19.1 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pathlib import Path
import sys

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('/content/drive/MyDrive/Thesis/BookSuccessPredictor/config.ini')

drive_base_path = Path(config['Drive']['drive_base_path'])

# sys.path.append(str(drive_base_path / 'BookSuccessPredictor' / '_utils'))
sys.path.append(str(drive_base_path / 'BookSuccessPredictor' / 'datasets' / 'goodreads_maharjan_super' / 'MultiModal' / 'dataset_loader'))

In [5]:
from MultimodalGoodreadsDataset import MultimodalGoodreadsDataset

In [6]:
dataset_base_dir = '/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/raw_preprocessed/goodreads_maharjan_trimmed'
cached_features_dir = '/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/MultiModal/dataset_loader/cached_features'

In [7]:
a = MultimodalGoodreadsDataset(dataset_base_dir, cached_features_dir)

Total test instances: 290, validation instances: 290, and Training instances: 404
Total unique books: 984
Training instances (404,), Val instances (290,), Test instances (290,)
extracting feature: char_5_gram
extracting feature: bert_features


# Get Transformer Model from Stage 1

In [ ]:
import wandb
run = wandb.init()

model_name = 'DistilBERT_multitask_overlap50_dataset_embeddings'

if config['Model']['name'] == 'distilbert-base-uncased':
  if (config['Tokenizer']['overlap']):
    artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/model-nlpbosie:v0', type='model')
  else:
    artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/model-2giwtwvy:v0', type='model')
    
artifact_dir = artifact.download()

transformer_model = DistilBERTForMultipleSequenceClassification.from_pretrained(artifact_dir, num_labels1 = 2, num_labels2 = 8)
model.cuda()

# Getting the Data

## Getting the Pooled Outputs

### Creating the Dataset

#### From script

In [ ]:
import torch as th
import time

def get_book_changes_idx(book_titles):
  book_changes_idx = np.where(np.array(book_titles[:-1]) != np.array(book_titles[1:]))[0]
  book_changes_idx += 1
  book_changes_idx = np.insert(book_changes_idx, 0, 0)
  return book_changes_idx

def getPooledOutputs(model, encoded_dataset, batch_size = 32):
  model.eval()

  # pooled_outputs = []
  pooled_outputs = torch.empty([0,768]).cuda()

  num_iters = (len(encoded_dataset['input_ids']) - 1)//batch_size + 1
  print("total number of iters ", num_iters)
  
  for i in range(num_iters):
    print(i)
    up_to = i*batch_size + batch_size
    if len(encoded_dataset['input_ids']) < up_to:
      up_to = len(encoded_dataset['input_ids'])
    input_ids = th.LongTensor(encoded_dataset['input_ids'][i*batch_size:up_to]).cuda()
    attention_mask = th.LongTensor(encoded_dataset['attention_mask'][i*batch_size:up_to]).cuda()

    with torch.no_grad():
      embeddings = model.forward(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)['hidden_states'][-1][:,0] # Pooled output
      pooled_outputs = th.cat([pooled_outputs, embeddings],0)
      th.cuda.empty_cache()

  return pooled_outputs

In [ ]:
chunked_encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'book_title', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 15752
    })
    validation: Dataset({
        features: ['attention_mask', 'book_title', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 11023
    })
    test: Dataset({
        features: ['attention_mask', 'book_title', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 10816
    })
})

In [ ]:
train_set_embeddings = getPooledOutputs(transformer_model, chunked_encoded_dataset['train'])

In [ ]:
val_set_embeddings = getPooledOutputs(transformer_model, chunked_encoded_dataset['validation'])

In [ ]:
test_set_embeddings = getPooledOutputs(transformer_model, chunked_encoded_dataset['test'])

In [ ]:
from datasets import Dataset
train_set_embeddings = Dataset.from_dict({'pooled_outputs': train_set_embeddings})
val_set_embeddings = Dataset.from_dict({'pooled_outputs': val_set_embeddings})
test_set_embeddings = Dataset.from_dict({'pooled_outputs': test_set_embeddings})

In [ ]:
from datasets import concatenate_datasets
dataset_w_embeddings = DatasetDict({
    'train': concatenate_datasets([chunked_encoded_dataset['train'], train_set_embeddings], axis = 1), 
    'validation': concatenate_datasets([chunked_encoded_dataset['validation'], val_set_embeddings], axis = 1), 
    'test': concatenate_datasets([chunked_encoded_dataset['test'], test_set_embeddings], axis = 1)
})
dataset_w_embeddings = dataset_w_embeddings.remove_columns(['attention_mask', 'input_ids', 'token_type_ids'])

In [ ]:
dataset_w_embeddings

DatasetDict({
    train: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 15752
    })
    validation: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 11023
    })
    test: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 10816
    })
})

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

with open('train_ds.pkl', 'wb') as output_file:
  pickle.dump(dataset_w_embeddings['train'], output_file)

with open('val_ds.pkl', 'wb') as output_file:
  pickle.dump(dataset_w_embeddings['validation'], output_file)

with open('test_ds.pkl', 'wb') as output_file:
  pickle.dump(dataset_w_embeddings['test'], output_file)

folder_id = '176pNJFvgTaclx_dKNGgocd-TvwmqxM7Y'
# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('train_ds.pkl')
file.Upload() 

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('val_ds.pkl')
file.Upload() 

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('test_ds.pkl')
file.Upload() 

#### Load from Drive

In [ ]:
base_path = Path("/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/")

In [ ]:
from datasets import DatasetDict

if model_name == 'DistilBERT_multitask_sentence_tokenized_dataset_embeddings':
  full_directory = base_path / 'DistilBERT_multitask_sentence_tokenized_dataset_embeddings'
elif model_name == 'DistilBERT_multitask_overlap50_dataset_embeddings':
  full_directory = base_path / 'DistilBERT_multitask_overlap50_dataset_embeddings'

with open(full_directory / 'train_ds.pkl', "rb") as input_file:
  train_set_embeddings = pickle.load(input_file)

with open(full_directory / 'val_ds.pkl', "rb") as input_file:
  val_set_embeddings = pickle.load(input_file)

with open(full_directory / 'test_ds.pkl', "rb") as input_file:
  test_set_embeddings = pickle.load(input_file)

dataset_w_embeddings = DatasetDict({'train': train_set_embeddings, 'validation': val_set_embeddings, 'test': test_set_embeddings})
dataset_w_embeddings

ay


DatasetDict({
    train: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 15752
    })
    validation: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 11023
    })
    test: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 10816
    })
})

## Average Pooled Outputs for Shallow Neural Network and SVM

### Generating the Data from Pooled Outputs

#### From Script

In [ ]:
def getAveragePooledOutputs(model, encoded_dataset):
  book_embeddings_dataset = {'meaned_pooled_output': [], 'book_title': [], 'genre': [], 'labels': []}

  book_changes = get_book_changes_idx(encoded_dataset['book_title'])

  for i in range(len(book_changes)):
    print(i)
    start = book_changes[i]
    end = None
    if i != len(book_changes) - 1:
      end = book_changes[i+1]
    else:
      end = len(encoded_dataset['input_ids'])

    input_ids = th.LongTensor(encoded_dataset['input_ids'][start:end])
    attention_mask = th.BoolTensor(encoded_dataset['attention_mask'][start:end])

    with torch.no_grad():
      embeddings = transformer_model.distilbert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)[0][:,0] # Pooled output
      book_embeddings = th.mean(embeddings, dim=0) # Takes the mean of the pooled output
    book_embeddings_dataset['meaned_pooled_output'].append(book_embeddings)
    book_embeddings_dataset['book_title'].append(encoded_dataset['book_title'][start])
    book_embeddings_dataset['genre'].append(encoded_dataset['genre'][start])
    book_embeddings_dataset['labels'].append(encoded_dataset['labels'][start])
  
  return book_embeddings_dataset

In [ ]:
avg_pld_outs_ds = getAveragePooledOutputs(dataset_w_embeddings)

In [ ]:
from datasets import DatasetDict, Dataset
avg_pld_outs_hf_ds = DatasetDict({'train': Dataset.from_dict(avg_pld_outs_ds['train']), 'validation': Dataset.from_dict(avg_pld_outs_ds['validation']), 'test': Dataset.from_dict(avg_pld_outs_ds['test'])})

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

with open('avg_pld_outs_hf_ds.pkl', 'wb') as output_file:
  pickle.dump(avg_pld_outs_hf_ds, output_file)

folder_id = '176pNJFvgTaclx_dKNGgocd-TvwmqxM7Y'
# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('avg_pld_outs_hf_ds.pkl')
file.Upload() 

#### Load from Drive

In [ ]:
!pip install datasets

     |████████████████████████████████| 542 kB 7.6 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 243 kB 52.8 MB/s 
     |████████████████████████████████| 118 kB 55.4 MB/s 


In [ ]:
from datasets import DatasetDict
with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
  avg_pld_outs_hf_ds = pickle.load(input_file)

ModuleNotFoundError: ignored

# Simple Shallow Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.modeling_outputs import SequenceClassifierOutput

class Net(nn.Module):

    def __init__(self, pre_classifier_init, classifier_init):
        super(Net, self).__init__()

        self.pre_classifier = nn.Linear(768, 768)
        self.classifier = nn.Linear(768, 2)
        self.dropout = nn.Dropout(0.1)

        self.pre_classifier.weight.data.copy_(pre_classifier_init.weight.data)
        self.classifier.weight.data.copy_(classifier_init.weight.data)

        # print(pre_classifier_init.bias.data)
        self.pre_classifier.bias.data.copy_(pre_classifier_init.bias.data)
        self.classifier.bias.data.copy_(classifier_init.bias.data)

        # DOUBLE CHECK IF BIASES ARE BEING SET AS WELL

    def forward(self, x, labels = None):
        # Max pooling over a (2, 2) window
        x = self.pre_classifier(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
          loss_fct = CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        return SequenceClassifierOutput(
            loss = loss,
            logits = logits
        )

net = Net(model.pre_classifier, model.classifier1)

#### Results with no Training

In [ ]:
net.eval()

In [ ]:
with torch.no_grad():
  logits = net.forward(torch.FloatTensor(avg_pld_outs_hf_ds['validation']['meaned_pooled_output']))
y_score = softmax(logits['logits'], axis = 1)[:, 1].tolist()
y_pred = [math.floor(input) if input < 0.50 else math.ceil(input) for input in y_score]
f1_score(avg_pld_outs_hf_ds['validation']['success_label'], y_pred, average = 'weighted')

In [ ]:
with torch.no_grad():
  logits = net.forward(torch.FloatTensor(avg_pld_outs_hf_ds['test']['meaned_pooled_output']))
y_score = softmax(logits['logits'], axis = 1)[:, 1].tolist()
y_pred = [math.floor(input) if input < 0.50 else math.ceil(input) for input in y_score]
f1_score(avg_pld_outs_hf_ds['test']['success_label'], y_pred, average = 'weighted')

#### Training w Hyperparameter Tuning and Results

In [ ]:
def load_data():
  with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
    avg_pld_outs_hf_ds = pickle.load(input_file)
  avg_pld_outs_hf_ds.set_format(type='pt', columns=['meaned_pooled_output', 'success_label'])
  trainset = avg_pld_outs_hf_ds['train']
  valset = avg_pld_outs_hf_ds['validation']
  return trainset, valset

def load_test_data():
  with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
    avg_pld_outs_hf_ds = pickle.load(input_file)
  avg_pld_outs_hf_ds.set_format(type='pt', columns=['meaned_pooled_output', 'success_label'])
  testset = avg_pld_outs_hf_ds['test']
  return testset

In [ ]:
from ray import tune
# from ray.tune.integration.wandb import wandb_mixin
# '''@wandb_mixin
# run = wandb.init()

def train_nn(config, checkpoint_dir, data_dir=None):
  net = Net(model.pre_classifier, model.classifier1, config['do_rate'])
  net.train()
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if torch.cuda.device_count() > 1:
          net = nn.DataParallel(net)
  print(type(net))
  net.to(device)
  # net.cuda()


  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

  trainset, valset = load_data()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=config["batch_size"], shuffle=True)
  valloader = torch.utils.data.DataLoader(valset, batch_size=config["batch_size"], shuffle=True)

  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):

      inputs = data['meaned_pooled_output']
      labels = data['success_label']

      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      epoch_steps += 1

      if i % 10 == 9:
        print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                        running_loss / epoch_steps))
        running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      net.eval()
      with torch.no_grad():
        for i, data in enumerate(valloader, 0):

          inputs_cpu = data['meaned_pooled_output']
          labels_cpu = data['success_label']

          inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
          # inputs.cuda()
          # labels.cuda()

          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)

          all_predictions = np.append(all_predictions, predicted.to('cpu').numpy())
          all_labels = np.append(all_labels, labels_cpu.numpy())

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          loss = criterion(outputs, labels)
          val_loss += loss.cpu().numpy()
          val_steps += 1

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((net.state_dict(), optimizer.state_dict()), path)

      net.train()

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      # s_acc = accuracy_score(all_labels, all_predictions)
      # wandb.log({"val_loss": val_loss / val_steps, "val_accuracy": correct / total})
      tune.report(loss=(val_loss / val_steps), accuracy=correct / total, f1=s_f1, precision=s_precision, recall=s_recall)
  print("Finished Training")

In [ ]:
def test_results(net, device="cpu"):
    testset = load_test_data()

    testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

    all_predictions = np.array([])
    all_labels = np.array([])

    net.eval()
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs_cpu = data['meaned_pooled_output']
            labels_cpu = data['success_label']

            inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_predictions = np.append(all_predictions, predicted.to('cpu').numpy())
            all_labels = np.append(all_labels, labels_cpu.numpy())

    s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
    return {
        'precision': s_precision,
        'recall': s_recall,
        'f1': s_f1
    }

In [ ]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.wandb import WandbLoggerCallback
import torch.optim as optim
from functools import partial

def main(num_samples = 15, max_num_epochs = 10):
  config = {
      "lr": tune.loguniform(1e-4, 1e-1),
      "batch_size": tune.choice([16,32,64,128]),
      "num_epochs": tune.choice([1,2]),#,2,3]),#,2,3,5,10,20]),
      "do_rate": tune.uniform(0.1, 0.5),
      "wandb": {
        "project": "AvgPooledOutputClassifier",
        "api_key": config['WandB']['api_key']
      }
    }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(train_nn, checkpoint_dir='/tmp/ShallowNNModels'),
    config = config,
    resources_per_trial={'gpu': 1},
    metric = 'loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="AvgPooledOutputClassifier",
        group='raytune_hpsearch',
        api_key=config['WandB']['api_key'],
        log_config=True
    )])
  
  best_trial = result.get_best_trial(metric="f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["accuracy"]))
  
  best_trained_model = Net(model.pre_classifier, model.classifier1, best_trial.config['do_rate'])
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      # if gpus_per_trial > 1:
      #     best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)

  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/Models/ShallowNNModels/yungclassifier1.pt"
  torch.save(best_trained_model.state_dict(), path)
  return test_results(best_trained_model, device)

In [ ]:
main(num_samples=15)

# SVM

In [ ]:
from sklearn import svm
import numpy as np

In [ ]:
cs = np.arange(0.9, 1.5, 0.02).tolist()
best_clf = None
best_score = 0
best_c = None
for c in cs:
  clf = svm.SVC(kernel='rbf', gamma='scale', C=c)
  clf.fit(avg_pld_outs_hf_ds['train']['meaned_pooled_output'], avg_pld_outs_hf_ds['train']['success_label'])
  predictions = clf.predict(avg_pld_outs_hf_ds['validation']['meaned_pooled_output'])
  (_, pred_counts) = np.unique(predictions, return_counts=True)
  val_score = f1_score(avg_pld_outs_hf_ds['validation']['success_label'], predictions, average = 'weighted')
  print('Clf with C = {} obtained val-score of {}'.format(c, val_score))
  if (val_score > best_score):
    best_score = val_score
    best_clf = clf
    best_c = c

print('\nBest C: {}; Val-score: {}'.format(best_c, best_score))
test_predictions = best_clf.predict(avg_pld_outs_hf_ds['test']['meaned_pooled_output'])
test_score = f1_score(avg_pld_outs_hf_ds['test']['success_label'], test_predictions, average = 'weighted')
print('Yields score of {} on test set'.format(test_score))

# RoBERT

In [ ]:
dataset_w_embeddings.set_format('pytorch', columns=['pooled_outputs', 'success_label', 'genre'])

In [ ]:
import numpy as np
from datasets import DatasetDict, Dataset

def get_book_changes_idx(book_titles):
  book_changes_idx = np.where(np.array(book_titles[:-1]) != np.array(book_titles[1:]))[0]
  book_changes_idx += 1
  # book_changes_idx = np.append(book_changes_idx, len(book_titles))
  # book_changes_idx = np.insert(book_changes_idx, 0, 0)
  return book_changes_idx

def convert_to_LSTM_dataset_full(dataset):
  full_ds = {}
  full_ds['train'] = convert_to_LSTM_dataset_sub(dataset['train'])
  full_ds['validation'] = convert_to_LSTM_dataset_sub(dataset['validation'])
  full_ds['test'] = convert_to_LSTM_dataset_sub(dataset['test'])

  full_ds = DatasetDict({'train': Dataset.from_dict(full_ds['train']), 'validation': Dataset.from_dict(full_ds['validation']), 'test': Dataset.from_dict(full_ds['test'])})
  # full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  return full_ds

def convert_to_LSTM_dataset_sub(dataset):
  ds = {'grouped_pooled_outs': None, 'success_label': None, 'genre': None}

  book_start_idx = get_book_changes_idx(dataset['book_title'])
  book_start_idx_w_end = np.append(book_start_idx, len(dataset['book_title']))
  book_start_idx_w_zero = np.insert(book_start_idx, 0, 0)

  book_lengths = book_start_idx_w_end - np.concatenate((np.array([0]), np.roll(book_start_idx_w_end, 1)[1:]))
  # print(type(dataset['pooled_outputs']))
  book_grouped_embeddings = dataset['pooled_outputs'].split_with_sizes(list(book_lengths))
  # book_grouped_embeddings = torch.stack(dataset['pooled_outputs'].split_with_sizes(list(book_lengths)), dim=0)

  # print(type(book_grouped_embeddings))
  ds['grouped_pooled_outs'] = book_grouped_embeddings
  ds['success_label'] = np.take(dataset['success_label'], book_start_idx_w_zero)
  ds['genre'] = np.take(dataset['genre'], book_start_idx_w_zero)
  return ds

In [ ]:
class RoBERT_Model(nn.Module):

    def __init__(self, layer_size = 100):
        self.layer_size = layer_size
        super(RoBERT_Model, self).__init__()
        self.lstm = nn.LSTM(768, layer_size, num_layers=1, bidirectional=False)
        self.out = nn.Linear(layer_size, 2)

    def forward(self, grouped_pooled_outs):
        """ Define how to performed each call
        Parameters
        __________
        pooled_output: array
            -
        lengt: int
            -
        Returns:
        _______
        -
        """
        # chunks_emb = pooled_out.split_with_sizes(lengt) # splits the input tensor into a list of tensors where the length of each sublist is determined by lengt

        seq_lengths = torch.LongTensor([x for x in map(len, grouped_pooled_outs)]) # gets the length of each sublist in chunks_emb and returns it as an array

        batch_emb_pad = nn.utils.rnn.pad_sequence(grouped_pooled_outs, padding_value=-91, batch_first=True) # pads each sublist in chunks_emb to the largest sublist with value -91
        batch_emb = batch_emb_pad.transpose(0, 1)  # (B,L,D) -> (L,B,D)
        lstm_input = nn.utils.rnn.pack_padded_sequence(batch_emb, seq_lengths, batch_first=False, enforce_sorted=False) # seq_lengths.cpu().numpy()

        packed_output, (h_t, h_c) = self.lstm(lstm_input, )  # (h_t, h_c))
#         output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, padding_value=-91)

        h_t = h_t.view(-1, self.layer_size) # (-1, 100)

        return self.out(h_t) # logits

In [ ]:
def my_collate1(batches):
  # for batch in batches:
  #   print(type(batch['grouped_pooled_outs']), len(batch['grouped_pooled_outs']))
  #   print(type(torch.FloatTensor(batch['grouped_pooled_outs'])))
    return {
        'grouped_pooled_outs': [torch.stack(x['grouped_pooled_outs']) for x in batches],
        'success_label': torch.LongTensor([x['success_label'] for x in batches])
    }

In [ ]:
from transformers import AdamW
import time

def load_test_data():
  full_ds = convert_to_LSTM_dataset_full(dataset_w_embeddings)
  full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  testset = full_ds['test']
  return testset

def load_data():
  full_ds = convert_to_LSTM_dataset_full(dataset_w_embeddings)
  full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  trainset = full_ds['train']
  valset = full_ds['validation']
  return trainset, valset

def loss_fun(outputs, targets):
    loss = nn.CrossEntropyLoss()
    return loss(outputs, targets)

def rnn_train_fun1(config, checkpoint_dir='/tmp/LSTMModels'):
  model = RoBERT_Model(config["layer_size"])
  model.train()
  device = "cpu"
  # if torch.cuda.is_available():
  #   device = "cuda:0"
  #   if torch.cuda.device_count() > 1:
  #       model = nn.DataParallel(model)
  # # print(type(model))
  # model.to(device)


  criterion = nn.CrossEntropyLoss()
  # optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
  optimizer=AdamW(model.parameters(), lr=config["lr"])

  trainset, valset = load_data()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=config["batch_size"], collate_fn=my_collate1)
  valloader = torch.utils.data.DataLoader(valset, batch_size=config["batch_size"], collate_fn=my_collate1)

  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0

    for batch_idx, batch in enumerate(trainloader):
      grouped_pooled_outs = batch['grouped_pooled_outs'] # .to(device)
      targets = batch['success_label'] #.to(device)

      optimizer.zero_grad()
      outputs = model(grouped_pooled_outs)
      loss = loss_fun(outputs, targets)
      loss.backward()
      model.float()
      optimizer.step()

      running_loss += loss.item()
      epoch_steps += 1

      if batch_idx % 10 == 9:
        print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                        running_loss / epoch_steps))
        running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      with torch.no_grad():
          for i, data in enumerate(valloader, 0):

              grouped_pooled_outs = data['grouped_pooled_outs'] # .to(device)
              targets = data['success_label'] # .to(device)

              outputs = model(grouped_pooled_outs)
              _, predicted = torch.max(outputs.data, 1)

              all_predictions = np.append(all_predictions, predicted.numpy())
              all_labels = np.append(all_labels, targets.numpy())

              loss = criterion(outputs, targets)
              val_loss += loss.cpu().numpy()
              val_steps += 1

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((model.state_dict(), optimizer.state_dict()), path)

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      tune.report(loss=(val_loss / val_steps), f1=s_f1, precision=s_precision, recall=s_recall)

In [ ]:
def test_results(net, device="cpu"):
  testset = load_test_data()
  testloader = torch.utils.data.DataLoader(testset, batch_size=8, collate_fn=my_collate1)

  all_predictions = np.array([])
  all_labels = np.array([])

  net.eval()
  with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        grouped_pooled_outs = data['grouped_pooled_outs'] # .to(device)
        targets = data['success_label'] # .to(device)

        outputs = net(grouped_pooled_outs)
        _, predicted = torch.max(outputs.data, 1)

        all_predictions = np.append(all_predictions, predicted.numpy())
        all_labels = np.append(all_labels, targets.numpy())

  s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

  return {
      'precision': s_precision,
      'recall': s_recall,
      'f1': s_f1
  }

In [ ]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger
from ray.tune.schedulers import ASHAScheduler
from functools import partial

def main(num_samples = 6, max_num_epochs = 15):
  config = {
    "lr": tune.loguniform(5e-4, 5e-2),
    "batch_size": tune.choice([16,32,64]),
    "num_epochs": tune.choice([1,2,3,5]),
    "layer_size": tune.choice([100]),
    "wandb": {
      "project": "LSTMClassifier",
      "api_key": config['WandB']['api_key'],
      "log_config": True
    }
  }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(rnn_train_fun1, checkpoint_dir='/tmp/LSTMModels'),
    config = config,
    resources_per_trial={'gpu': 1},
    metric = 'loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="LSTMClassifier",
        group='raytune_hpsearch',
        api_key=config['WandB']['api_key'],
        log_config=True
    )])

  
  best_trial = result.get_best_trial(metric="f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["f1"]))
  
  best_trained_model = RoBERT_Model(best_trial.config['layer_size'])
  device = "cpu"
  # if torch.cuda.is_available():
  #     device = "cuda:0"
      # if gpus_per_trial > 1:
      #     best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)
                        
  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/Models/LSTMModels/yungclassifier1.pt"
  torch.save(best_trained_model.state_dict(), path)
  return test_results(best_trained_model, device)

In [ ]:
test_results = main()

# MultiModal

Our model will be composed of three separate modules:

1. (Normalizer) Responsible for taking all the inputs of various dimensions and feeding them each through their own linear layer to project them into a space with all the same dimensions

In essence, it is responsible for eq (1) in the paper $h_i=selu(W_{h_i} x_i + b_h)$


2. (GenreAwareAttention) This is where most of the meat of the model is. It is responsible for performing these 3 equations. 

$score(h_i, g) = v^T selu(W_a h_i + W_g g + b_a)$

$\alpha_i = \frac{exp(score(h_i,g))}{\sum_{i'}exp(score(h_{i'},g)}$

$r=\sum_i \alpha_i h_i$

3. (ClassOutput) The last layer is simply responsible for projecting the book representation to class probabilities.

$\hat{p}=\sigma(W_c r + b_c)$

In [ ]:
class GenreAwareAttention(nn.Module):
  def __init__(self, num_units):
    super(GenreAwareAttention, self).__init__()
    self.activation = nn.SELU()
    self.nn_softmax = nn.Softmax(dim=1)

    self.v = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(num_units,1)),
        requires_grad=True
    )

    self.Wa = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(num_units,num_units)), 
        requires_grad=True
    )

    self.b = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(num_units,1)),
        requires_grad=True
    )

    self.Wg = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(8, num_units)), 
        requires_grad=True
    )

  def forward(self, input):
    # x ~ (BATCH_SIZE, NUM_MODALITIES, EMBED_SIZE)
    # g ~ (BATCH_SIZE, 1, GENRE_EMBED_SIZE)
    x, g = input
    
    # calculate scores
    atten_g = torch.mm(g, self.Wg)
    et = self.activation(torch.matmul(x, self.Wa) + atten_g + self.b)
    et = torch.matmul(et, v)

    at = self.nn_softmax(et)

    at = torch.unsqueeze(at, axis=-1)
    ot = at * x

    return torch.sum(ot, axis=1) # BATCH_SIZE, EMBED_SIZE

In [2]:
import torch
import torch.nn as nn

In [40]:
W_g = torch.randn(8,50) # (GENRE_EMBED_SIZE, NUM_UNITS)
b = torch.randn(50,)
W_a = torch.randn(100,50) # (EMBED_SIZE, NUM_UNITS)
v = torch.randn(50,)

g = torch.randn(2,1,8)
atten_g = torch.matmul(g,W_g)

x = torch.randn(2,2,100) # (BATCH_SIZE, NUM_MODALITIES, EMBED_SIZE)
atten_x = torch.matmul(x,W_a)

et = atten_x + atten_g + b

activation = nn.SELU()
et = activation(et)

et = torch.matmul(et, v) # (BATCH_SIZE, NUM_MODALITIES)

nn_softmax = nn.Softmax(dim=1)
at = nn_softmax(et)

at = torch.unsqueeze(at, axis=-1)
ot = at * x

return torch.sum(ot, axis=1) # BATCH_SIZE, EMBED_SIZE

In [41]:
at = torch.unsqueeze(at, axis=-1)

In [30]:
print("at", at.size())
print("x", x.size())
ot = at * x

at torch.Size([2, 2, 1])
x torch.Size([2, 2, 100])


In [50]:
ot.size()

torch.Size([2, 2, 100])

tensor([[ 2.2919e-01, -1.8977e-01,  1.7535e-01, -2.2514e-01, -7.6712e-02,
         -1.9979e-01, -1.1810e+00,  5.3156e-01, -3.0408e-01,  3.4963e-01,
          1.3997e+00,  9.7225e-01, -5.4858e-01,  5.7299e-01, -7.7324e-01,
         -9.1313e-01, -4.1739e-01, -8.0439e-01,  2.3484e+00, -2.9007e-01,
         -1.2715e+00, -1.8511e+00, -2.8565e-01,  6.7159e-01,  8.5598e-01,
         -9.8786e-01,  5.6124e-01,  2.3515e+00, -1.8396e+00,  5.1102e-01,
         -1.4442e+00, -1.7484e-01, -1.4984e+00,  2.0399e+00, -4.6582e-01,
          2.1591e-01, -1.3654e+00, -6.6240e-01,  1.2415e+00, -1.7705e-01,
         -1.3034e+00,  2.6808e-01, -1.7598e+00, -3.1317e-01, -1.1997e+00,
         -1.3368e+00,  1.3313e+00,  2.4553e+00,  1.6137e-01,  3.6597e-01,
         -1.1853e+00, -4.8756e-02,  1.8581e-04,  3.3820e-01,  7.5581e-02,
         -1.0947e+00, -1.6166e+00,  5.1022e-01,  5.1499e-01,  9.3621e-01,
         -7.9039e-03, -4.4066e-01,  3.0480e-01,  9.8368e-01, -6.2125e-02,
          4.9277e-01, -1.3250e+00, -1.

In [47]:
at = torch.Tensor([[[0.1],[0.9]],[[0.6],[0.4]]])
x_ones = torch.ones(2,2,100)

In [48]:
at.size()

torch.Size([2, 2, 1])

In [49]:
at*x_ones

tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000, 0.1000, 0.1000],
         [0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9

In [32]:
ot.size()

torch.Size([2, 2, 100])

In [3]:
at = torch.Tensor([[0,1],[1,1]])
nn_softmax = nn.Softmax(dim=1)
nn_softmax(et)

tensor([[0.2689, 0.7311],
        [0.5000, 0.5000]])

In [9]:
torch.empty(2,1)

tensor([[-3.9840e+00],
        [ 3.0669e-41]])

In [65]:
atten_g + b

tensor([[[-1.5261, -0.2775,  0.8154,  0.4122, -6.9972,  1.2513, -4.5939,
          -2.5548,  2.2302, -1.2488,  0.8603,  0.5032, -3.8734,  1.8478,
           1.4973, -3.2242,  1.7572,  0.6788, -1.2209, -1.1413,  2.9790,
          -1.7827,  1.5457,  3.1319, -2.6864, -2.2457,  0.7975,  3.6945,
          -2.2696, -0.5522,  0.5278, -3.4796,  1.2006,  3.7119,  1.0593,
           2.7440,  4.4950,  5.4137,  0.5259, -6.2668, -1.9521,  1.3269,
           3.7726, -0.0122, -4.4477, -1.4995,  2.1863, -4.4230,  3.5518,
          -3.5179]],

        [[-0.4361,  0.2425, -0.8461, -1.9067, -1.6588,  0.6462,  3.5050,
          -3.6282,  4.5438,  1.6389,  1.0426,  8.3637, -0.4873, -0.4006,
           1.3407,  1.8167, -0.0745,  3.0692, -2.5674,  1.0390, -0.6313,
          -2.4083, -4.5563,  6.0730,  5.9005,  0.0382, -3.2180,  0.4446,
          -2.5942,  1.0744,  7.4257,  0.3901,  2.3321,  1.6781, -2.6127,
          -1.5325,  4.1785,  2.4973,  3.0282,  3.3333,  3.3621, -2.2427,
           3.5387, -3.3777,  

In [62]:
atten_g.size()

torch.Size([2, 1, 50])

In [57]:
torch.matmul(x,W_a).size()

torch.Size([13, 2, 50])

In [55]:
x + W_a

RuntimeError: ignored

In [49]:
torch.mm(x[0],W_a)

tensor([[ -5.2253,  -2.1407,  24.4201,  -3.9515, -12.0565,   8.9632,  -5.8922,
           9.6811,  -1.4307,   3.4983, -11.7638,  -9.9432, -26.8291,   0.5046,
          -6.5346,   6.4835, -13.0093,   1.0672, -14.8226,  -5.2222,  18.0818,
          -7.5220,  -8.1664,   8.7183, -28.4626,   2.5043,   0.4199,  -1.2806,
          -6.0073,  11.4323,   6.6603, -12.9509,  -6.1603,  -6.8321,   5.9607,
          -1.9642,  20.1633,  14.3140,  20.1914,  19.4655,  -3.2368,  -4.0109,
          -8.8130,  18.9713,  16.3852, -25.0359,  19.3567,  21.4507,  15.5199,
          -6.8079, -13.7123,   4.4378,  -8.1074,  13.3745,   7.2898,   5.1411,
           8.7620,  13.0857,  16.3901,  -0.8492, -11.7379,   9.7090,  -8.4135,
         -33.2244, -19.0405,  -0.3070, -26.2229, -29.9931,  26.7524,  25.7659,
          -7.9887,   9.5237,  18.9548,  12.9002,  24.2067,  -9.8002,   4.5973,
           4.7791,  -4.4266, -16.6876,   0.9315,  -2.7920,  11.1141,  25.8602,
          17.1112,   2.9836,  18.6475,  34.2724,  27

In [ ]:
class Normalizer(nn.Module):
  def __init__(self):
    super(Normalizer, self).__init__()

    self.c5g_linear = nn.Linear(None, None)
    self.bf_linear = nn.Linear(None, None)

  def forward(self, input)
    # split features into char_5_gram and bert_features
    char_5_grams = None
    bert_features = None

    c5g_normed = self.c5g_linear(char_5_grams)
    bf_normed = self.bf_linear(bert_features)

    # concatenate c5g_normed and bf_normed
    features_normed = None
    return features_normed